### ATAC enrichment

In [1]:
library(data.table)
# library(qvalue)
library(bedr)
options(scipen = 999)



######################
#### bedr v1.0.7 ####
######################

checking binary availability...
  * Checking path for bedtools... PASS
    /home/ssm-user/anaconda3/envs/single_cell/bin/bedtools
  * Checking path for bedops... FAIL
  * Checking path for tabix... FAIL
tests and examples will be skipped on R CMD check if binaries are missing




### Copy gracie's attempt with memento  eqtls

In [ ]:
for (ct in c('T4', 'cM', 'ncM', 'T8', 'B', 'NK')){
print(paste('starting....',ct))
library(data.table)
library(qvalue)
library(bedr)
options(scipen = 999)
# args=commandArgs(TRUE)

res_raw=data.frame(fread(paste('/data_volume/memento/lupus/full_analysis/memento/100kb/eur_',ct, '.csv', sep='')))
#res_raw=data.frame(fread(args[1]))[-1]
#emp=fread(args[2])
#delta=fread(args[3])
out=paste('/data_volume/memento/lupus/atac_enrichment/100kb/', 'eur_memento_',ct, '.out', sep='')
#expr_genes=read.table(args[3],header=F)

print('gene')
#print(head(expr_genes))
#print(dim(expr_genes))
#print(head(emp))
#emp$FDR=qvalue(emp$Empirical)$qvalues
#thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

#run if not corrected by celltype
#fdr=qvalue(res_raw$pvalue)
#print(fdr)
#res_raw$FDR=fdr$qvalues

#filter FDR
#res=res_raw[res_raw$FDR <= 0.1,]
#snps gene statistic pvalue FDR beta qvalue


#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
#take in metasoft data, create atac enrichments from fixed effects
colnames(res_raw)=c('SNP', 'gene', 'statistic', 'p.value', 'FDR', 'beta')
# res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
#res=res_raw
#filter for expressed genes
# print(head(res_raw))
# print(dim(res_raw))
#res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
#print(dim((res_expr)))
#res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

# print('expr')
#print(dim(res_expr))
#print(head(res_expr))
# print('no gene')
#print(head(res_noexpr))

#num_no=length(res_noexpr$gene)
#print(num_no)
#nullp=runif(num_no, min = 0, max = 1)
#nullp=rep(1,num_no)
#res_noexpr$p.value=nullp
#res=rbind(res_expr,res_noexpr)
#print(head(res_expr))
#print(dim(res_expr))
res=res_raw
# print(dim(res))
# print(tail(res))
#atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
#atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

#names(atac)=c('chr', 'start', 'end', 'group')
#atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
atac.groups=names(table(atac$group))
#atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

#res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

#res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
#res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
#res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

all.df=NULL
for(g in c('B', 'myeloid', 'nk', 'T')){
    print(g)
    peaks=atac[which(atac$group == g), ]
    peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
    overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
    in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
    pvals=res$"p.value"[match(res.bed.sorted, res$id)]
    #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
    #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
    mw_res=wilcox.test(pvals ~ in_peak, alternative='greater')
    df=data.frame(W=mw_res$statistic, pval=mw_res$p.value, group=g)
    all.df=rbind(all.df, df)   
}

write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')

}

In [6]:
atac.groups

[1] "B"          "EPI"        "myeloid"    "nk"         "open"      
[6] "progenitor" "T"          "thymocyte"

In [5]:
conf.int

W,pval,conf,estimate,group
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
4590296557,0.767464,-0.006153402,-0.001645391,B
4590296557,0.767464,Inf,-0.001645391,B


### Copy gracie's attempt with merged  eqtls

In [ ]:
for (pop in c('asian')){
    for (ct in c('T4', 'cM', 'ncM', 'T8', 'B', 'NK')){
        print(ct)
        library(data.table)
        library(qvalue)
        library(bedr)
        options(scipen = 999)
        # args=commandArgs(TRUE)

        res_raw=data.frame(fread(paste('/data_volume/memento/lupus/atac_enrichment/merged_eqtls/',pop,'_',ct, '_merged.tsv', sep='')))
        #res_raw=data.frame(fread(args[1]))[-1]
        #emp=fread(args[2])
        #delta=fread(args[3])
        out=paste('/data_volume/memento/lupus/atac_enrichment/gracie/', pop, '_',ct, '_merged.out', sep='')
        #expr_genes=read.table(args[3],header=F)

        print('gene')
        #print(head(expr_genes))
        #print(dim(expr_genes))
        #print(head(emp))
        #emp$FDR=qvalue(emp$Empirical)$qvalues
        #thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

        #run if not corrected by celltype
        #fdr=qvalue(res_raw$pvalue)
        #print(fdr)
        #res_raw$FDR=fdr$qvalues

        #filter FDR
        #res=res_raw[res_raw$FDR <= 0.1,]
        #snps gene statistic pvalue FDR beta qvalue


        #colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
        #take in metasoft data, create atac enrichments from fixed effects
        colnames(res_raw)=c('SNP', 'gene', 'statistic_mem', 'p-value_mem', 'FDR_mem', 'beta_mem', 'beta_meq', 't-stat_meq', 'p-value_meq', 'FDR_meq')
        # res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

        #colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
        #res=res_raw
        #filter for expressed genes
        print(head(res_raw))
        print(dim(res_raw))
        #res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
        #print(dim((res_expr)))
        #res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

        print('expr')
        #print(dim(res_expr))
        #print(head(res_expr))
        print('no gene')
        #print(head(res_noexpr))

        #num_no=length(res_noexpr$gene)
        #print(num_no)
        #nullp=runif(num_no, min = 0, max = 1)
        #nullp=rep(1,num_no)
        #res_noexpr$p.value=nullp
        #res=rbind(res_expr,res_noexpr)
        #print(head(res_expr))
        #print(dim(res_expr))
        res=res_raw
        print(dim(res))
        print(tail(res))
        #atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
        #atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
        atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

        #names(atac)=c('chr', 'start', 'end', 'group')
        #atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
        atac.groups=names(table(atac$group))
        #atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

        res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
        res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

        #res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
        #res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
        res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
        res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

        #res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
        #res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
        #res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
        #res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

        all.df=NULL
        for(g in atac.groups){
            print(g)
            peaks=atac[which(atac$group == g), ]
            peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
            overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
            in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
            pvals_mem=res$"p-value_mem"[match(res.bed.sorted, res$id)]
            pvals_meq=res$"p-value_meq"[match(res.bed.sorted, res$id)]
            #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
            #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
            mw_res_mem=wilcox.test(pvals_mem ~ in_peak, alternative='greater')
            mw_res_meq=wilcox.test(pvals_meq ~ in_peak, alternative='greater')

            df=data.frame(W_mem=mw_res_mem$statistic, W_meq=mw_res_meq$statistic, pval_mem=mw_res_mem$p.value, pval_meq=mw_res_meq$p.value,group=g)
            all.df=rbind(all.df, df)   
        }

        write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')

    }
}

### Copy gracie's attempt with mateqtl  eqtls

In [ ]:
for (ct in c('B','T8', 'T4', 'NK', 'ncM', 'cM')){
print(ct)
library(data.table)
library(qvalue)
library(bedr)
options(scipen = 999)
# args=commandArgs(TRUE)

res_raw=data.frame(fread(paste('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/eur_',ct, '_all_hg19.csv', sep='')))
#res_raw=data.frame(fread(args[1]))[-1]
#emp=fread(args[2])
#delta=fread(args[3])
out=paste('/data_volume/memento/lupus/atac_enrichment/100kb/', 'eur_matqetl_',ct, '.out', sep='')
#expr_genes=read.table(args[3],header=F)

print('gene')
#print(head(expr_genes))
#print(dim(expr_genes))
#print(head(emp))
#emp$FDR=qvalue(emp$Empirical)$qvalues
#thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

#run if not corrected by celltype
#fdr=qvalue(res_raw$pvalue)
#print(fdr)
#res_raw$FDR=fdr$qvalues

#filter FDR
#res=res_raw[res_raw$FDR <= 0.1,]
#snps gene statistic pvalue FDR beta qvalue


#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
#take in metasoft data, create atac enrichments from fixed effects
colnames(res_raw)=c('SNP', 'gene', 'beta','statistic', 'p.value', 'FDR')
# res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

#colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
#res=res_raw
#filter for expressed genes
print(head(res_raw))
print(dim(res_raw))
#res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
#print(dim((res_expr)))
#res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

print('expr')
#print(dim(res_expr))
#print(head(res_expr))
print('no gene')
#print(head(res_noexpr))

#num_no=length(res_noexpr$gene)
#print(num_no)
#nullp=runif(num_no, min = 0, max = 1)
#nullp=rep(1,num_no)
#res_noexpr$p.value=nullp
#res=rbind(res_expr,res_noexpr)
#print(head(res_expr))
#print(dim(res_expr))
res=res_raw
print(dim(res))
print(tail(res))
#atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
#atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

#names(atac)=c('chr', 'start', 'end', 'group')
#atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
atac.groups=names(table(atac$group))
#atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

#res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

#res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
#res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
#res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
#res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

all.df=NULL
for(g in atac.groups){
    print(g)
    peaks=atac[which(atac$group == g), ]
    peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
    overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
    in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
    pvals=res$"p.value"[match(res.bed.sorted, res$id)]
    #pvals=res$"pvalue"[match(res.bed.sorted, res$id)]
    #pvals=res$"p-value"[match(res.bed.sorted, res$id)]
    mw_res=wilcox.test(pvals ~ in_peak, alternative='greater')
    df=data.frame(W=mw_res$statistic, pval=mw_res$p.value, group=g)
    all.df=rbind(all.df, df)   
}

write.table(all.df, file=out, row.names=F, col.names=T, quote=F, sep='\t')

}


### Get confidence interval

In [5]:
matches <- c(
    'T4'='T',
    'cM'='myeloid',
    'ncM'='myeloid',
    'B'='B',
    'NK'='nk'
)

In [10]:
all.df=NULL
pop <- 'asian'
for (ct in c('T4', 'cM', 'ncM', 'T8', 'B', 'NK')){
    print(ct)
    library(data.table)
    library(qvalue)
    library(bedr)
    options(scipen = 999)
    # args=commandArgs(TRUE)

#     res_raw=data.frame(fread(paste('/data_volume/memento/lupus/full_analysis/memento/100kb/', pop, '_',ct, '.csv', sep='')))
    res_raw=data.frame(fread(paste('/data_volume/memento/lupus/full_analysis/mateqtl/outputs/', pop, '_',ct, '_all_hg19.csv', sep='')))

    #res_raw=data.frame(fread(args[1]))[-1]
    #emp=fread(args[2])
    #delta=fread(args[3])
    #expr_genes=read.table(args[3],header=F)

    print('gene')
    #print(head(expr_genes))
    #print(dim(expr_genes))
    #print(head(emp))
    #emp$FDR=qvalue(emp$Empirical)$qvalues
    #thresh=max(emp$Pvalue[which(emp$FDR < 0.1)])

    #run if not corrected by celltype
    #fdr=qvalue(res_raw$pvalue)
    #print(fdr)
    #res_raw$FDR=fdr$qvalues

    #filter FDR
    #res=res_raw[res_raw$FDR <= 0.1,]
    #snps gene statistic pvalue FDR beta qvalue


    #colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta")
    #take in metasoft data, create atac enrichments from fixed effects
    colnames(res_raw)=c('SNP', 'gene', 'beta','statistic', 'p.value', 'FDR')
    # res_raw$gene=sapply(strsplit(res_raw$RSID, '_'), '[', 2)

    #colnames(res_raw)=c("SNP","gene","statistic","p.value","FDR","beta","qvalue")
    #res=res_raw
    #filter for expressed genes
    print(head(res_raw))
    print(dim(res_raw))
    #res_expr=res_raw[res_raw$gene %in% expr_genes$V1,]
    #print(dim((res_expr)))
    #res_noexpr=res_raw[!(res_raw$gene %in% expr_genes$V1),]

    print('expr')
    #print(dim(res_expr))
    #print(head(res_expr))
    print('no gene')
    #print(head(res_noexpr))

    #num_no=length(res_noexpr$gene)
    #print(num_no)
    #nullp=runif(num_no, min = 0, max = 1)
    #nullp=rep(1,num_no)
    #res_noexpr$p.value=nullp
    #res=rbind(res_expr,res_noexpr)
    #print(head(res_expr))
    #print(dim(res_expr))
    res=res_raw
    print(dim(res))
    print(tail(res))
    #atac=fread('/ye/yelabstore2/10x.lupus/process.scRNAseq/eqtls/S9_lineage_groups.txt.gz')
    #atac=fread('simple_sorted_atac_lineage_groups2.txt.gz')
    atac=fread('/data_volume/memento/lupus/atac_enrichment/sorted_simple_atac_lineage_groups3.bed.gz')

    #names(atac)=c('chr', 'start', 'end', 'group')
    #atac.bed=data.frame(chr=atac$chr, start=atac$start, end=atac$stop, group=atac$group)
    atac.groups=names(table(atac$group))
    #atac.sorted=bedr.sort.region(atac$chr,atac$start,atac$stop)

    res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
    res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))

    #res$chr=paste('chr', sapply(strsplit(res$snp, ':'), '[', 1), sep='')
    #res$pos=as.numeric(sapply(strsplit(res$snp, ':'), '[', 2))
    res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
    res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))

    #res$chr=paste('chr', sapply(strsplit(res$SNP, ':'), '[', 1), sep='')
    #res$pos=as.numeric(sapply(strsplit(res$SNP, ':'), '[', 2))
    #res$id=paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')
    #res.bed.sorted=bedr.sort.region(unique(paste(res$chr, ':', res$pos -1, '-' ,res$pos, sep='')))
    
    if (ct == 'T8'){
        groups <- c('T', 'nk')
    }
    else {
        groups <- c(matches[ct])
    }

    for(g in groups){
        print(g)
        peaks=atac[which(atac$group == g), ]
        peaks=paste(peaks$chr, ':', peaks$start, '-', peaks$stop, sep='')
        overlap.sig.snps=bedr(input=list(a=res.bed.sorted, b=peaks), method='intersect')
        in_peak=as.numeric(res.bed.sorted %in% overlap.sig.snps)
        pvals=res$"p.value"[match(res.bed.sorted, res$id)]
        write.table(
            cbind(in_peak, pvals),
            file=paste('/data_volume/memento/lupus/atac_enrichment/matched/mateqtl/', pop, '_',ct, '_', g,'.txt', sep=''), 
            row.names=F, col.names=T, quote=F, sep='\t')
        
    }
    

}
# write.table(all.df, file='asian_memento_confint.txt', row.names=F, col.names=T, quote=F, sep='\t')


[1] "T4"
[1] "gene"
          SNP  gene       beta statistic
1 12:56435929 RPS26 -0.9567400 -29.05693
2 12:56403577 RPS26  1.0081453  25.35022
3 12:56412487 RPS26  1.0081453  25.35022
4 12:56416928 RPS26  1.0081453  25.35022
5 12:56427808 RPS26  1.0081453  25.35022
6 12:56435412 RPS26  0.9745304  25.22708
                                        p.value
1 0.0000000000000000000000000000000000004792371
2 0.0000000000000000000000000000000009385490465
3 0.0000000000000000000000000000000009385490465
4 0.0000000000000000000000000000000009385490465
5 0.0000000000000000000000000000000009385490465
6 0.0000000000000000000000000000000012263871276
                                      FDR
1 0.0000000000000000000000000000008482765
2 0.0000000000000000000000000003322568742
3 0.0000000000000000000000000003322568742
4 0.0000000000000000000000000003322568742
5 0.0000000000000000000000000003322568742
6 0.0000000000000000000000000003617956489
[1] 1770056       6
[1] "expr"
[1] "no gene"
[1] 1770056       